In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd
import random
import torch
import time
import numpy as np
from gensim.models.word2vec import Word2Vec
from model import BatchProgramClassifier
from torch.autograd import Variable
from torch.utils.data import DataLoader

import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.autograd import Variable

import re

from pycparser.c_ast import TypeDecl, ArrayDecl
from pycparser import c_ast

In [2]:
root = '/home/david/projects/university/astnn/data/'

word2vec = Word2Vec.load(root+"train/embedding/node_w2v_128").wv
embeddings = np.zeros((word2vec.vectors.shape[0] + 1, word2vec.vectors.shape[1]), dtype="float32")
embeddings[:word2vec.vectors.shape[0]] = word2vec.vectors

HIDDEN_DIM = 100
ENCODE_DIM = 128
LABELS = 104
EPOCHS = 15
BATCH_SIZE = 1
USE_GPU = False
MAX_TOKENS = word2vec.vectors.shape[0]
EMBEDDING_DIM = word2vec.vectors.shape[1]

model = BatchProgramClassifier(EMBEDDING_DIM,HIDDEN_DIM,MAX_TOKENS+1,ENCODE_DIM,LABELS,BATCH_SIZE,
                               USE_GPU, embeddings)
model.load_state_dict(torch.load("/home/david/projects/university/astnn/model.pt"))

<All keys matched successfully>

# Load Data

In [3]:
word2vec = Word2Vec.load('/home/david/projects/university/astnn/data/train/embedding/node_w2v_128').wv
vocab = word2vec.vocab

ast_data = pd.read_pickle(root+'test/test_.pkl')
block_data = pd.read_pickle(root+'test/blocks.pkl')

# Allowed var names

In [4]:
leaf_embed = nn.Sequential(
    model._modules['encoder']._modules['embedding'],
    model._modules['encoder']._modules['W_c']
)

In [5]:
# words we wont allow as variable names
reserved_words = [
    'auto',
    'break',
    'case',
    'char',
    'const',
    'continue',
    'default',
    'do',
    'int',
    'long',
    'register',
    'return',
    'short',
    'sizeof',
    'static',
    'struct',
    'switch',
    'typedef',
    'union',
    'unsigned',
    'void',
    'volatile',
    'while',
    'double',
    'else',
    'enum',
    'extern',
    'float',
    'for',
    'goto',
    'if',
    'printf',
    'scanf',
    'cos',
    'malloc'
]


def allowed_variable(var):
    pattern = re.compile("([a-z]|[A-Z]|_)+([a-z]|[A-Z]|[0-9]|_)*$")
    if (var not in reserved_words) and pattern.match(var):
        return True
    else:
        return False

allowed_variable('scanf')

False

In [6]:
embedding_map = {}

for index in range(len(vocab)):
    if allowed_variable(word2vec.index2word[index]):
        embedding_map[index] = leaf_embed(torch.tensor(index)).detach().numpy()

# Var replace functions

In [7]:
def replace_index(node, old_i, new_i):
    i = node[0]
    if i == old_i:
        result = [new_i]
    else:
        result = [i]
    children = node[1:]
    for child in children:
        result.append(replace_index(child, old_i, new_i))
    return result

def replace_var(x, old_i, new_i):
    mod_blocks = []
    for block in x:
        mod_blocks.append(replace_index(block, old_i, new_i))

    return mod_blocks

# Closest Var functions

In [8]:
def l1_norm(a, b):
    return np.linalg.norm(a-b, 1)

def l2_norm(a, b):
    return np.linalg.norm(a-b)

def cos_sim(a, b):
    return np.inner(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def closest_index(embedding, embedding_map, metric):
    embedding = embedding.detach().numpy()
    closest_i = list(embedding_map.keys())[0]
    closest_dist = metric(embedding_map[closest_i], embedding)
    for i, e in embedding_map.items():
        d = metric(embedding_map[i], embedding)
        if d < closest_dist:
            closest_dist = d
            closest_i = i
    return closest_i

def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return v / norm

# Grad locating functions

In [9]:
def get_embedding(indices, node_list):
    '''
    get the embeddings at the index positions in postorder traversal.
    '''
    res = []
    c = 0
    for i in range(node_list.size(0)):
        if not np.all(node_list[i].detach().numpy() == 0):
            if c in indices:
                res.append(node_list[i])
            c += 1
    return res

def post_order_loc(node, var, res, counter):
    '''
    
    '''
    index = node[0]
    children = node[1:]
    for child in children:
        res, counter = post_order_loc(child, var, res, counter)
    if var == index and (not children):
        res.append(counter) 
#         print(counter, word2vec.index2word[index])
    counter += 1
    return res, counter

def get_grad(x, var_index, node_list):
    grads = []
    for i, block in enumerate(x):
        indices, _ = post_order_loc(block, var_index, [], 0)
        grads += get_embedding(indices, node_list.grad[:, i, :])
        try:
            node_embedding = get_embedding(indices, node_list[:, i, :])[0]
        except:
            pass
        
    if len(grads) < 1:
        return None, None
    grad = torch.stack(grads).sum(dim=0)
    return grad, node_embedding

# Var name finder

In [10]:

class declarationFinder(c_ast.NodeVisitor):
    def __init__(self):
        self.names = set()
    
    def visit_Decl(self, node):
        if type(node.type) in [TypeDecl, ArrayDecl] :
            self.names.add(node.name)

def get_var_names(ast):
    declaration_finder = declarationFinder()
    declaration_finder.visit(ast)
    return declaration_finder.names
    
# get_var_names(x)

# FGSM

with vars ordered and early exit

In [11]:
# def gradient_method(x, n_list, var, epsilon, metric):

#     orig_index = vocab[var].index if var in vocab else MAX_TOKEN

#     grad, node_embedding = get_grad(x, orig_index, n_list)
#     if grad is None:
# #         print("no leaf occurences")
#         return None

#     v = node_embedding.detach().numpy()
#     g = torch.sign(grad).detach().numpy()
    

#     v = v + epsilon * g
#     # get the closest emebedding from our map
#     i = closest_index(v, sampled_embedding_map, metric)
# #         print("orig name:", word2vec.index2word[orig_index], "; new name:", word2vec.index2word[i])
#     if i != orig_index:
#         return replace_var(x, orig_index, i)
#     else:
#         return x

In [12]:
MAX_TOKEN = word2vec.vectors.shape[0]

In [13]:
import time
import datetime

def evaluate(epsilon, limit = None, sort_vars = True):
    ast_count = 0
    var_count = 0

    ast_total = 0
    var_total = 0

    start = time.time()
    for code_id in block_data['id'].tolist():
    #     print(code_id)
        x, ast = block_data['code'][code_id], ast_data['code'][code_id]

        _, orig_pred = torch.max(model([x]).data, 1)
        orig_pred = orig_pred.item()

        # get the grad
        loss_function = torch.nn.CrossEntropyLoss()
        labels = torch.LongTensor([orig_pred])
        output = model([x])
        loss = loss_function(output, Variable(labels))
        loss.backward()
        n_list = model._modules['encoder'].node_list

        var_names = get_var_names(ast)
        success = False
        
        var_weighted = []
        for var in list(var_names):
            
            orig_index = vocab[var].index if var in vocab else MAX_TOKEN
            grad, node_embedding = get_grad(x, orig_index, n_list)
            if grad is not None:
                h = abs((grad @ torch.sign(grad)).item())
                var_weighted.append( (h, grad, node_embedding) )
            
        if sort_vars:
            var_weighted = sorted(var_weighted, key=lambda x: x[0], reverse = True)
        
        for h, grad, node_embedding in var_weighted:
            
            v = node_embedding
            g = torch.sign(grad)


            v = v + epsilon * g
            # get the closest emebedding from our map
#             i = closest_index(v, sampled_embedding_map, l2_norm)
            i = closest_index(v, sampled_embedding_map, cos_sim)
            if i != orig_index:
                new_x_l2 = replace_var(x, orig_index, i)
            else:
                new_x_l2 = x
            
            if new_x_l2:
                o = model([new_x_l2])
                _, predicted_l2 = torch.max(o.data, 1)

    #             print(orig_pred, predicted_l2.item())
                var_total += 1
                if orig_pred != predicted_l2.item():
                    var_count += 1
                    success = True
                    break

        if success:
            ast_count += 1
        ast_total += 1


        if ast_total % 500 == 499:
            eval_time = time.time() - start
            eval_time = datetime.timedelta(seconds=eval_time)
            print(ast_total, ";", eval_time, ";", ast_count / ast_total, ";", var_count / var_total)
    
        if limit and limit < ast_total:
            break
    return (ast_count / ast_total, var_count / var_total)


In [14]:
# sample_rate = 0.2
# sample_count = int(len(embedding_map) * sample_rate)
# sampled_embedding_map = {key: embedding_map[key] for key in random.sample(embedding_map.keys(), sample_count)}

sampled_embedding_map = embedding_map

In [15]:
evaluate(10)

499 ; 0:05:18.702358 ; 0.06613226452905811 ; 0.010655473038424281
999 ; 0:11:50.678249 ; 0.06906906906906907 ; 0.01107011070110701
1499 ; 0:18:57.640237 ; 0.07538358905937291 ; 0.012034078807241746
1999 ; 0:26:48.284370 ; 0.07703851925962982 ; 0.012212529738302934
2499 ; 0:36:03.751547 ; 0.07603041216486595 ; 0.011997979287698914
2999 ; 0:45:53.149422 ; 0.07535845281760586 ; 0.011884728649558267
3499 ; 0:55:05.345233 ; 0.07630751643326665 ; 0.012091296078253782
3999 ; 1:03:42.916828 ; 0.07476869217304326 ; 0.011816313626304142
4499 ; 1:10:11.879108 ; 0.07535007779506557 ; 0.011945873564028472
4999 ; 1:17:21.407472 ; 0.07561512302460492 ; 0.011951057573745613
5499 ; 1:24:01.683448 ; 0.07619567194035279 ; 0.012050964940032789
5999 ; 1:30:50.509424 ; 0.07634605767627937 ; 0.012086346123396844
6499 ; 1:37:27.652857 ; 0.07631943375903985 ; 0.01208312017345124
6999 ; 1:44:19.539827 ; 0.0757251035862266 ; 0.011974694984184365
7499 ; 1:51:04.838778 ; 0.07614348579810641 ; 0.012043110539303566


(0.07585809056821459, 0.012021391677966876)